In [1]:
import os
import pandas as pd
import lightgbm as lgb
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
import wandb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, root_mean_squared_error, r2_score
from dotenv import load_dotenv
import matplotlib.pyplot as plt

/home/brupesh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
load_dotenv()
os.environ["WANDB_API_KEY"] = os.getenv("WANDB_API_KEY")
os.environ["WANDB_DIR"] = "../data/wandb_logs"
wandb.login(key=os.environ["WANDB_API_KEY"])

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/brupesh/.netrc
wandb: Currently logged in as: brupeshmit (brupeshmit-massachusetts-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
train = pd.read_parquet("../data/train_clean_v2.parquet")
test = pd.read_parquet("../data/test_clean_v2.parquet")
target_col = "label"
y = train[target_col]
X = train.drop(columns=[target_col])

test_ids = test.iloc[:, 0]
test = test.drop(columns=[test.columns[0]])

feature_df = pd.read_csv("../data/lgbm_feature_importance.csv")
top_features = feature_df.sort_values(by="importance", ascending=False)["feature"].iloc[:400].tolist()
X = X[top_features]
test = test[top_features]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val)

In [15]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 512),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "feature_pre_filter": False,
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,
        "verbosity": -1
    }

    try:
        callbacks = [lgb.log_evaluation(100)]
        if params["boosting_type"] != "dart":
            callbacks.append(lgb.early_stopping(50))

        model = lgb.train(
            params,
            dtrain,
            num_boost_round=500,
            valid_sets=[dval],
            callbacks=callbacks
        )

        preds = model.predict(X_val)

        rmse = root_mean_squared_error(y_val, preds)

        r2 = r2_score(y_val, preds)
        pearson = pd.Series(preds).corr(pd.Series(y_val))

        wandb.log({
            "rmse": rmse,
            "r2": r2,
            "pearson": pearson
        }, step=trial.number)

        return rmse

    except Exception as e:
        trial.set_user_attr("fail_reason", str(e))
        wandb.log({"error": str(e)}, step=trial.number)
        raise


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, callbacks=[WeightsAndBiasesCallback()])

[I 2025-06-15 22:13:40,528] A new study created in memory with name: no-name-c3038646-5122-495b-8f18-344de253420b
/tmp/ipykernel_18869/510708740.py:56: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  study.optimize(objective, n_trials=30, callbacks=[WeightsAndBiasesCallback()])


error,got an unexpected ke...


[100]	valid_0's rmse: 1.01827
[200]	valid_0's rmse: 1.01542
[300]	valid_0's rmse: 1.0125
[400]	valid_0's rmse: 1.00971


[I 2025-06-15 22:13:50,130] Trial 0 finished with value: 1.0067197373909311 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.0022052205989138525, 'num_leaves': 223, 'max_depth': 4, 'min_child_samples': 41, 'subsample': 0.7956842513166731, 'colsample_bytree': 0.9919147170610438, 'reg_alpha': 3.6922237808659315e-07, 'reg_lambda': 0.00023403658282525685}. Best is trial 0 with value: 1.0067197373909311.


[500]	valid_0's rmse: 1.00672
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.388399
[200]	valid_0's rmse: 0.266684
[300]	valid_0's rmse: 0.225553
[400]	valid_0's rmse: 0.204693
[500]	valid_0's rmse: 0.19215
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 0.19215


...

[I 2025-06-15 22:25:56,164] Trial 27 finished with value: 0.16752715496339976 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.1216210872190907, 'num_leaves': 242, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.8087414094590983, 'colsample_bytree': 0.6815878769865791, 'reg_alpha': 1.6693672407810182e-08, 'reg_lambda': 4.4301121123123475e-05}. Best is trial 25 with value: 0.16439155452148407.


Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.749003
[200]	valid_0's rmse: 0.607193
[300]	valid_0's rmse: 0.511802
[400]	valid_0's rmse: 0.442226
[500]	valid_0's rmse: 0.391288
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 0.391288


[I 2025-06-15 22:26:13,653] Trial 28 finished with value: 0.39128818917115965 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.02274983188021961, 'num_leaves': 178, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.7404429979405185, 'colsample_bytree': 0.6211104705010171, 'reg_alpha': 1.1805374761908769e-07, 'reg_lambda': 4.0634880687554775e-05}. Best is trial 25 with value: 0.16439155452148407.


Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.874631
[200]	valid_0's rmse: 0.755605
[300]	valid_0's rmse: 0.672363
[400]	valid_0's rmse: 0.611517
[500]	valid_0's rmse: 0.561993
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 0.561993


[I 2025-06-15 22:26:34,904] Trial 29 finished with value: 0.5619932693046067 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.009237271183881266, 'num_leaves': 198, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.8205044991543983, 'colsample_bytree': 0.7664113394305357, 'reg_alpha': 5.833037824832554e-07, 'reg_lambda': 0.00010229437758048311}. Best is trial 25 with value: 0.16439155452148407.


In [16]:
print(f"Best trial number: {study.best_trial.number}")
print(f"Best RMSE: {study.best_value}")
print("Best trial params:")
for k, v in study.best_trial.params.items():
    print(f"  {k}: {v}")

Best trial number: 25
Best RMSE: 0.16439155452148407
Best trial params:
  boosting_type: gbdt
  learning_rate: 0.13391504838227442
  num_leaves: 242
  max_depth: 11
  min_child_samples: 15
  subsample: 0.8103913798475878
  colsample_bytree: 0.7633005751835771
  reg_alpha: 1.4310064470041852e-07
  reg_lambda: 0.0011779585848832991


In [ ]:
best_params = study.best_trial.params
best_params.update({"objective": "regression", "metric": "rmse", "device": "gpu", "gpu_platform_id": 0, "gpu_device_id": 0})
model = lgb.train(
    best_params,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dval],
    callbacks=[
        lgb.early_stopping(50),
        lgb.log_evaluation(100)
    ]
)

model.save_model("../data/lightgbm_opt_model_1.txt")

Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.300522
[200]	valid_0's rmse: 0.216325
[300]	valid_0's rmse: 0.187
[400]	valid_0's rmse: 0.172423
[500]	valid_0's rmse: 0.164564
[600]	valid_0's rmse: 0.159373
[700]	valid_0's rmse: 0.155883
[800]	valid_0's rmse: 0.153271
[900]	valid_0's rmse: 0.151295
[1000]	valid_0's rmse: 0.149602
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.149602


In [ ]:
preds = model.predict(test)
pd.DataFrame({"id": test_ids, "prediction": preds}).to_csv("../data/lightgbm_opt1_preds.csv", index=False)

wandb.finish()

In [8]:
print()